# Sionna Ray Tracing Visualization

This notebook loads the room scene with tables, chairs, door, and window, then performs ray tracing visualization with Sionna.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from sionna.rt import load_scene, Camera, Transmitter, Receiver, PlanarArray
import matplotlib.pyplot as plt
import IPython.display

In [2]:
# Load the scene
scene_path = '/tf/Project_1/room_5x3x3.xml'

print(f"Loading scene from: {scene_path}")
scene = load_scene(scene_path)

# Configure the scene for ray tracing
scene.frequency = 2.4e9  # 2.4 GHz (WiFi frequency)
scene.synthetic_array = True

print("✓ Scene loaded and configured.")
print(f"Objects in scene: {list(scene.objects.keys())}")

scene.preview(show_devices=False)

Loading scene from: /tf/Project_1/room_5x3x3.xml


RuntimeError: ​[xml.cpp:179] Error while loading "/tf/Project_1/room_5x3x3.xml" (at line 4, col 6): unexpected tag "radio_material".

In [3]:
# Assign radio materials for RF simulation
print("Assigning radio materials...")
print(f"Available objects in scene: {list(scene.objects.keys())}")

# Map XML IDs to Sionna Radio Materials
# Based on your XML: 
# - 'mat_white' (walls, ceiling, pillar) -> itu_concrete
# - 'mat_wood' (furniture, door, furniture_center) -> itu_wood
material_map = {
    'floor': 'itu_concrete',
    'ceiling': 'itu_concrete',        # or 'itu_plasterboard' if it's a false ceiling
    'walls': 'itu_concrete',
    'pillar': 'itu_concrete',
    'window': 'itu_glass',
    'door': 'itu_wood',
    'furniture': 'itu_wood',
    'furniture_center': 'itu_wood',   # <--- Added this (was missing)
    'led_tv': 'itu_metal'
}

assigned = []
for obj_name, material in material_map.items():
    obj = scene.get(obj_name)
    if obj is not None:
        obj.radio_material = material
        assigned.append(f"  - {obj_name}: {material.replace('itu_', '')}")
    else:
        # This helps debug if there's a typo in the XML ID
        print(f"  ⚠ Warning: Object '{obj_name}' defined in map but not found in scene")

print("✓ Materials assigned successfully:")
for msg in assigned:
    print(msg)

Assigning radio materials...
Available objects in scene: ['floor', 'ceiling', 'walls', 'window', 'door', 'furniture', 'pillar', 'furniture_center', 'led_tv']
✓ Materials assigned successfully:
  - floor: concrete
  - ceiling: concrete
  - walls: concrete
  - pillar: concrete
  - window: glass
  - door: wood
  - furniture: wood
  - furniture_center: wood
  - led_tv: metal


In [4]:
# Add a camera for visualization
# Room: 5m(x) x 3m(y) x 3m(z)
# Position camera inside room, looking at center
cam_pos = [1.0, 1.5, 2.0]  # Inside the room
cam_look_at = [2.5, 1.5, 1.0]  # Looking at center area
camera = Camera("viz_camera", position=cam_pos, look_at=cam_look_at)
scene.add(camera)
print(f"✓ Camera added at {cam_pos}, looking at {cam_look_at}")

✓ Camera added at [1.0, 1.5, 2.0], looking at [2.5, 1.5, 1.0]


In [5]:
# Configure antenna arrays for transmitter and receiver
scene.tx_array = PlanarArray(num_rows=1, num_cols=1, 
                            vertical_spacing=0.5, 
                            horizontal_spacing=0.5, 
                            pattern="iso", 
                            polarization="V")

scene.rx_array = PlanarArray(num_rows=1, num_cols=1, 
                            vertical_spacing=0.5, 
                            horizontal_spacing=0.5, 
                            pattern="iso", 
                            polarization="V")

# Add transmitter (near ceiling)
tx_pos = [2.5, 1.5, 2.8]
tx = Transmitter("tx", tx_pos)
scene.add(tx)

# Add receiver (near table level)
rx_pos = [1.0, 0.5, 0.8]
rx = Receiver("rx", rx_pos)
scene.add(rx)

print(f"✓ Transmitter at {tx_pos}")
print(f"✓ Receiver at {rx_pos}")

✓ Transmitter at [2.5, 1.5, 2.8]
✓ Receiver at [1.0, 0.5, 0.8]


In [6]:
# Compute ray paths for RF propagation analysis
print("\nComputing ray paths...")
paths = scene.compute_paths(max_depth=3, num_samples=1e6)

print(f"✓ Ray tracing complete")
print(f"Number of paths computed: {paths.mask.shape}")
print(f"Path types captured: Reflection, Diffraction, Scattering")


Computing ray paths...
✓ Ray tracing complete
Number of paths computed: (1, 1, 1, 54)
Path types captured: Reflection, Diffraction, Scattering


In [7]:
# Visualize paths in 3D
print("Generating 3D ray visualization...")
scene.preview(paths=paths, show_devices=True, show_paths=True)

Generating 3D ray visualization...


Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, matrixWorldNeedsUpda…